地理信息系统设计与开发（第一次课）

**json写入shp**

In [1]:
# !pip install ogr
# !pip install osr

In [2]:
from osgeo import gdal
from osgeo import ogr# 矢量处理数据包(shp,json等)
from osgeo import osr# 投影坐标包
from osgeo import gdal_array
from osgeo import gdalconst

import json# json是存储和交换文本信息的语法，类似 XML
import requests# 网络数据获取(开源数据，不开源就用爬虫)
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
url="https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.geojson"

In [4]:
try:
  r = requests.get(url,timeout=30)
  r.raise_for_status()
  r.encoding = r.apparent_encoding
  print(requests.head(url).headers)
  print(r.text[0:100])
except:
  print("error!")

{'Content-Type': 'application/json; charset=utf-8', 'Connection': 'keep-alive', 'Date': 'Mon, 05 Jun 2023 04:31:40 GMT', 'Server': 'Apache', 'Cache-Control': 'public, max-age=60', 'Expires': 'Mon, 05 Jun 2023 04:32:40 GMT', 'Last-Modified': 'Mon, 05 Jun 2023 04:31:40 GMT', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Methods': '*', 'Access-Control-Allow-Headers': 'accept,origin,authorization,content-type', 'X-Frame-Options': 'SAMEORIGIN', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1', 'Strict-Transport-Security': 'max-age=31536000', 'Vary': 'Accept-Encoding', 'X-Cache': 'Hit from cloudfront', 'Via': '1.1 b47618c03bd47cf085f27b1e215f76cc.cloudfront.net (CloudFront)', 'X-Amz-Cf-Pop': 'SEA73-P2', 'X-Amz-Cf-Id': 'eC-ZorZkQqq-v-0kdrenFT6JcBlpq0IzeA-cVoOZkCI5zj7QYHCyyw==', 'Age': '45'}
{"type":"FeatureCollection","metadata":{"generated":1685939501000,"url":"https://earthquake.usgs.gov


In [5]:
# 通过ogr的GetDriverByName创建driver
driver = ogr.GetDriverByName('ESRI Shapefile')

In [6]:
# 通过driver的CreatDataSource创建datasource
ds = driver.CreateDataSource('myEarthQuake.shp')

In [7]:
# 通过datasource的CreateLayer方法创建layer
# ## 创建空间参考spatial_reference
spatial_reference = osr.SpatialReference()
spatial_reference.ImportFromEPSG(4326)
layer = ds.CreateLayer('myEarthQuake',spatial_reference,geom_type=ogr.wkbPoint)

In [8]:
# 通过layer的CreateField方法创建属性表结构
layer.CreateField(ogr.FieldDefn('magnitude',ogr.OFTReal))
layer.CreateField(ogr.FieldDefn('place',ogr.OFTString))
layer.CreateField(ogr.FieldDefn('time',ogr.OFTReal))
layer.CreateField(ogr.FieldDefn('id',ogr.OFTString))
layer.CreateField(ogr.FieldDefn('longitude',ogr.OFTReal))
layer.CreateField(ogr.FieldDefn('latitude',ogr.OFTReal))
layer.CreateField(ogr.FieldDefn('depth',ogr.OFTReal))

0

In [10]:
# 通过json获取相关数据
geojson = json.loads(r.text)
features = geojson['features']
import matplotlib
print('earthquakepoint num:',len(features))

# print(features[0:1])
for feat in features:
  properties = feat['properties']
  # print(properties)
  magnitude = properties['mag']
  place = properties['place']
  time = properties['time']
  geom = feat['geometry']
  longitude = geom['coordinates'][0]
  latitude = geom['coordinates'][1]
  depth = geom['coordinates'][2]
  # print(geom)

  # 通过layer的GetlayerDefn方法获取feature的属性表结构
  feature_defn = layer.GetLayerDefn()

  # 通过OGR创建新的feature
  feature = ogr.Feature(feature_defn)

  # 设定feature中的相关字段值
  feature.SetField('magnitude',magnitude)
  feature.SetField('place',place)
  feature.SetField('time',time)
  feature.SetField('longitude',longitude)
  feature.SetField('latitude',latitude)
  feature.SetField('depth',depth)

  # 通过OGR的Geometry类创建point对象
  point = ogr.Geometry(ogr.wkbPoint)#类型

  # 通过point的AddPoint方法设定坐标
  point.AddPoint(longitude,latitude)

  # 通过feature的SetGeometry方法将feature设定为新建的Point
  feature.SetGeometry(point)

  # 通过layer的CreatrFeature方法将point feature添加到layer
  layer.CreateFeature(feature)

  fig = plt.figure(figsize=(10,10),dpi=300)
  plt.plot(longitude,latitude,'o',markersize=2,color='k')
  matplotlib.pyplot.close()





earthquakepoint num: 11384


In [13]:
#展示图形
plt.show()

In [14]:
del ds